# Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением — введение регуляризации. Ее можно вводить несколькими способами.

## L1, L2 регуляризации

Мы уже разбирали самый простой способ — добавление штрафа к весам в функцию потерь. На сходимость нейросети это, правда, влияет слабо.

$$ \text{Loss_reg} = \text{loss} + \lambda \cdot \text{reg}$$

$$ \text{reg}_{L1} = \lambda \sum |w_i| $$

$$ \text{reg}_{L2} = \lambda \sum w_i^2 $$

<center><img src ="https://ml.gan4x4.ru/msu/dep-2.2/L07/loss_landscape_with_regularization.gif" width="800"></center>

<center><em>Source: <a href="https://people.eecs.berkeley.edu/~jrs/189/">Introduction to Machine Learning
</a></em></center>


Иногда уже его хватает, чтобы решить все проблемы. Напомним, что **L2** Loss приводит к большому числу маленьких ненулевых весов в сети. А **L1** Loss — к маленькому числу ненулевых весов (разреженной нейросети).

В PyTorch L2-регуляризация (или её аналоги) часто "встроена" в оптимизаторы и связана с параметром `weight_decay`. Подробнее различия между `weight_decay` и L2 мы обсудим ниже.


```
sgd = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)
```



## Dropout

Одним из распространенных именно в нейросетях методом регуляризации является **Dropout**.

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/dropout.png" width="700"></center>

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью $p$ **зануляем выход нейронов** слоя (например, $p = 0.5$).
2. Зануленные нейроны не участвуют в данном `forward`, и поэтому градиент к ним при `backward` не идет.
3. Сила регуляризации определяется вероятностью $p$: чем она больше, тем сильнее регуляризация.

###Сверточные слои

Аналогично **Batch Normalization**, при применении к **сверточному слою**  **Dropout** должен **убирать каналы целиком**. Dropout для полносвязного слоя реализован в PyTorch в `nn.Dropout` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html), для сверточного — в `nn.Dropout2d` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html).

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/dropout_2d.png" width="700"></center>

### Мотивация Dropout

#### Борьба с коадаптацией

Одной из  проблем при работе с глубокими сетями является **совместная адаптация нейронов**. Если все весовые коэффициенты обучаются вместе, некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

**Часть нейронов** делает основную работу — **предсказывает**, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина: один нейрон делает **неверное предсказание**, другие его **исправляют**, и в итоге первый нейрон свои ошибки не исправляет.

Можно попробовать понять, как это работает, через аналогию — конвейер по сборке машины: основную часть работы может исполнять небольшое количество работников. Другие работники могут лениться и передавать деталь, которая к ним поступила, без изменений или откручивать только что добавленные части. Если мы исключим “эффективных работников”, результат станет удручающим.

Это явление называется **коадаптацией**. Его нельзя было предотвратить с помощью традиционной регуляризации, такой как **L1** или **L2**. А вот **Dropout** с этим хорошо борется.

Отключая хорошо обученные нейроны, мы заставляем плохо обученные нейроны учиться. Отключая нейроны, которые исправляют ошибки других нейронов, мы заставляем ошибающиеся нейроны исправлять ошибки.

**Dropout** гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со **случайно выбранными подмножествами функций**, выученных другими нейронами, улучшая **обобщающую способность** нейронов.

На следующем рисунке, извлеченном из [статьи про Dropout 🎓[article]](https://jmlr.org/papers/v15/srivastava14a.html), мы находим сравнение признаков, изученных в наборе данных **MNIST** нейросетью с **одним скрытым слоем** в автоэнкодере, имеющем $256$ признаков после ReLU **без Dropout** (слева), и признаков, изученных той же структурой с использованием **Dropout** в ее скрытом слое с $p = 0.5$ (справа).

Первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать. Второй явно демонстрирует целенаправленное распределение весов, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<center><img src ="https://ml.gan4x4.ru/msu/dep-2.2/L07/compare_weights_with_dropout_and_without_dropout.png" width="600"></center>

<center><em>Source: <a href="https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf">Dropout: A Simple Way to Prevent Neural Networks from
Overfitting</a></em></center>

#### Dropout как регуляризация

Фактически **Dropout** штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть **Dropout**, мы сообщаем ей, что решение, которое мы ожидаем, должно быть устойчиво к шуму.

#### Dropout как ансамбль

Можно рассматривать **Dropout** как **ансамбль нейросетей** со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, [выключая Dropout в режиме eval ✏️[blog]](https://habr.com/ru/companies/wunderfund/articles/330814/).

Таким образом, возникает аналогия со случайным лесом: каждая из наших нейросетей легко выучивает выборку и переобучается — имеет низкий bias, но высокий variance. При этом, за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance.



#### Dropout помогает бороться с переобучением

Dropout, в силу указанного выше, может хорошо помогать бороться с переобучением.

И в случае линейных слоев:

<center><img src ="https://ml.gan4x4.ru/msu/dep-2.2/L07/dropout_solve_overfitting_problem_in_mlp_networks.png" width="500">

<em>Source: <a href="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/">Tutorial: Dropout as Regularization and Bayesian Approximation</a></em></center>

И в случае свёрточных слоёв:

<center><img src ="https://ml.gan4x4.ru/msu/dep-2.2/L07/dropout_solve_overfitting_problem_in_convolution_networks.png" width="500">

<em>Source: <a href="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/">Tutorial: Dropout as Regularization and Bayesian Approximation</a></em></center>

#### Простая реализация Dropout

Напишем "наивную" реализацию модуля Dropout.

**Замечание:** этот блок кода дан для объяснения работы Dropout. При построении нейронной сети используйте `nn.Dropout` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) или `nn.Dropout2d` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html).

```
class BadDropout(nn.Module):
    def __init__(self, p: float=0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}"
            )
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(device)
            return x * keep
        # in test time, expectation is calculated
        return x * (1 - self.p)
```



Приведенная реализация неоптимальна, так как в режиме инференса (когда `training = False`) функция `forward` совершает дополнительное умножение. Одним из приоритетов при создании модели является скорость работы в режиме инференса. Поэтому по возможности все "лишние" операции выполняют только в режиме обучения. В данном случае можно целиком убрать коэффициент нормировки из режима инференса, перенеся его в режим обучения в знаменатель.

Дополнительным плюсом такого подхода является то, что при удалении модуля из архитектуры сети функция прогнозирования не изменится.


```
class Dropout(nn.Module):
    def __init__(self, p: float=0.2):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}"
            )
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(x)
            return x * keep / (1 - self.p)
        return x  # in test time, expectation is calculated intrinsically - we just not divide weights
```



Попробуем применить Dropout в нашей нейросети:

In [ ]:
class SimpleMNIST_NN_Dropout(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid, init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(nn.Dropout(p=0.2))  # add Dropout
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

Так как наша модель из-за Dropout ведет себя по-разному во время обучения и во время тестирования, мы должны прямо ей сообщать, обучается она сейчас или нет.

$$
\begin{array}{c|c}
\large{\text{model.train()}}&\ \large{\text{model.eval()}}\\  \hline
\large{\text{Активируются Dropout слои}}&\large{\text{Слои Dropout отключены}}\\
\large{\text{Выход части нейронов обнуляется, выходы нормируются}}&\large{\text{Все нейроны работают}}
\end{array}
$$



Обучим модель с **Dropout**:

In [ ]:
model_name = "nn3_dropout"
model = SimpleMNIST_NN_Dropout(n_layers=3)

trainer = L.Trainer(
    max_epochs=5,
    logger=TensorBoardLogger(save_dir=f"logs/{model_name}"),
    num_sanity_val_steps=0,
)

pipeline = Pipeline(model=model, exp_name=model_name, optimizer_kwargs={"lr": 1e-2})

trainer.fit(model=pipeline, train_dataloaders=train_loader, val_dataloaders=val_loader)

history = pipeline.history
history["epoсhs"] = trainer.max_epochs
history_plotter.add(history)

In [ ]:
history_plotter.plot(["n_layers3_sigmoid", "n3_layers_sigmoid_havier", model_name])

В данном случае выигрыш небольшой. Если учить нейросеть больше эпох, эффект будет заметнее.

#### Пример борьбы с переобучением при помощи Dropout


Чтобы увидеть эффект и при этом не учить нейросеть 100+ эпох, сделаем искусственный пример.

Просто **добавим к линейной зависимости шум** и попробуем выучить ее нейронной сетью.

[[colab] 🥨 Batch Normalization and Dropout](https://colab.research.google.com/github/Niranjankumar-c/DeepLearning-PadhAI/blob/master/DeepLearning_Materials/7_BatchNormalization/BatchNorm_Dropout.ipynb)

In [ ]:
N = 50  # number of data points
noise = 0.3

# generate the train data
x_train = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_train = x_train + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

# generate the test data
x_test = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_test = x_test + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

print(f"x_train shape: {x_train.shape}\nx_test shape: {x_test.shape}")

In [ ]:
plt.scatter(
    x_train.data.numpy(), y_train.data.numpy(), c="purple", alpha=0.5, label="train"
)
plt.scatter(
    x_test.data.numpy(), y_test.data.numpy(), c="yellow", alpha=0.5, label="test"
)

x_real = np.arange(-1, 1, 0.01)
y_real = x_real
plt.plot(x_real, y_real, c="green", label="true")
plt.legend()
plt.show()

Модель **без Dropout**:

In [ ]:
N_h = 100  # num of neurons
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Модель **с Dropout**:

In [ ]:
N_h = 100  # num of neurons

model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5),  # 50 % probability
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    nn.Dropout(0.2),  # 20% probability
    nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)

In [ ]:
num_epochs = 1500
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):
    # train without dropout
    y_pred = model(x_train)  # look at the entire data in a single shot
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # train with dropout
    y_pred_dropout = model_dropout(x_train)
    loss_dropout = criterion(y_pred_dropout, y_train)
    optimizer_dropout.zero_grad()
    loss_dropout.backward()
    optimizer_dropout.step()

    if epoch % 300 == 0:
        model.eval()  # not train mode
        model_dropout.eval()  #  not train mode

        # get predictions
        y_test_pred = model(x_test)
        test_loss = criterion(y_test_pred, y_test)

        y_test_pred_dropout = model_dropout(x_test)
        test_loss_dropout = criterion(y_test_pred_dropout, y_test)
        # plotting data and predictions
        plt.scatter(
            x_train.data.numpy(),
            y_train.data.numpy(),
            c="purple",
            alpha=0.5,
            label="train",
        )
        plt.scatter(
            x_test.data.numpy(),
            y_test.data.numpy(),
            c="yellow",
            alpha=0.5,
            label="test",
        )
        plt.plot(
            x_test.data.numpy(), y_test_pred.data.numpy(), "r-", lw=3, label="normal"
        )
        plt.plot(
            x_test.data.numpy(),
            y_test_pred_dropout.data.numpy(),
            "b--",
            lw=3,
            label="dropout",
        )

        plt.title(
            "Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f"
            % (epoch, test_loss, test_loss_dropout)
        )

        plt.legend()

        model.train()  # train mode
        model_dropout.train()  # train mode

        plt.pause(0.05)

Видим, что **нейросеть без Dropout сильно переобучилась**.

#### Доверительный интервал от Dropout

Dropout можно рассматривать как **ансамбль моделей**.

Если мы возьмем один и тот же объект и пропустим его через модель с **активным Dropout 1000 раз**, это будет эквивалентно тому, чтобы пропустить этот объект через **1000 моделей с различными конфигурациями**. Таким образом мы получим распределение предсказаний, на основе которого можно вычислять [**доверительные интервалы** 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D0%B2%D0%B5%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B0%D0%BB) и ловить аномалии (объекты с большой дисперсией предсказываемых значений, которые плохо распознаются сетью).

<center><img src ="https://ml.gan4x4.ru/msu/dep-2.2/L07/confidence_interval_dropout.png" width="600"></center>

<center><em>Source: <a href="https://atcold.github.io/NYU-DLSP20/en/week14/14-3/">Overfitting and regularization</a></em></center>



## DropConnect

Если занулять не нейроны (активации), а случайные веса с вероятностью $p$, получится DropConnect.

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/dropconnect.png" width="650"></center>

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель, но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения.

В принципе, вариантов зануления чего-то в нейронной сети можно предложить великое множество, в разных ситуациях будут работать разные способы ([в этом списке 🎓[article]](https://paperswithcode.com/methods/category/regularization) много Drop...).

## DropBlock

Например, можно убирать для каждого батча из нейросети случайные блоки из слоев. И это будет работать!

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/dropblock.png" width="750"></center>

[[arxiv] 🎓 Deep Networks with Stochastic Depth (Huang et al., 2016)](https://arxiv.org/pdf/1603.09382.pdf)

## Batch Normalization до или после Dropout



### После

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/batchnormalization_after_dropout.png" width="400"></center>

* **Плохо**: В режиме обучения **Dropout** будет отключать (занулять) выходы слоя активации, из-за этого накопленные в режиме обучения **значения скользящего среднего матожидания и дисперсии** для вычисления **Batch Normalization** в режиме тестирования будут иметь **сдвиг** (не компенсируемый приведением слоя Dropout в режим `train()`), который приведет к нестабильной работе нейросети.


Подробно:
* [[arxiv] 🎓 Understanding the Disharmony between Dropout and Batch Normalization by
Variance Shift (Li et al., 2018)](https://arxiv.org/pdf/1801.05134.pdf)
* [[git] 🐾 Understanding the Disharmony between Dropout and Batch Normalization](https://github.com/adelizer/kaggle-sandbox/blob/master/drafts/dropout_bn.ipynb)

### До

<center><img src ="https://ml.gan4x4.ru/msu/dev-2.1/L07/out/batchnormalization_before_dropout.png" width="400"></center>

* **Лучше**: Меньше влияние сдвига от **Dropout** на **BatchNorm**. Но если на предыдущих слоях есть **Dropout**, то значения все равно могут быть смещены.

### Ставить только что-то одно

* **Dropout** может отрицательно влиять на качество нейросети с **BatchNorm** за счет разного поведения на train и test. Лучше не ставить **Dropout** раньше **BatchNorm**.